# Chapter 12 – Custom Models and Training with TensorFlow

This notebook contains all the sample code and solutions to the exercises in chapter 12, as well as code examples from Appendix C

### Setup
This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

And TensorFlow ≥ 2.8:

In [2]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

## Using TensorFlow like NumPy
### Tensors and Operations
### Tensors